# Accessing Data from UNHCR

Input file: Syra + Refugee_1, provided from UNHCR  

In [1]:
import pandas as pd
import os
import re
import preprocessor
from ekphrasis.classes.tokenizer import Tokenizer
from nltk.tokenize.treebank import TreebankWordDetokenizer 
from ekphrasis.classes.segmenter import Segmenter
import pandas as pd
import re
import nltk
from tqdm.auto import tqdm

In [2]:
df = pd.read_csv("/Users/emiliatrulsson/Desktop/Syria Preprocess/Syra + Refugees_1.csv", delimiter=';', header=0, low_memory=False)

In [3]:
df.head(5)

,Date,Headline,URL,Opening Text,Hit Sentence,Source,Influencer,Country,Subregion,Language,...,Twitter Screen Name,User Profile Url,Twitter Bio,Twitter Followers,Twitter Following,Alternate Date Format,Time,State,City,Document Tags
0,15-Jan-2011 11:58PM,称阿拉克重水反应堆生产出氘化物 伊朗公布一项核进展,http://www.tianjinwe.com/rollnews/201101/t2011...,... 据新华社德黑兰1月15日电 （记者 何光海 杜源江）伊朗代理外长兼原子能组织主席萨利...,... 。他说：“阿拉克重水反应堆设施在氘化物方面已经实现自给自足。” 受伊朗政府邀请，阿尔...,天津网,何光海 杜源江,Mainland China,Tianjin,Chinese Simplified,...,NaN,NaN,NaN,NaN,NaN,15-jan-11,11:58 em,NaN,NaN,NaN
1,15-Jan-2011 11:57PM,「阪神大震災１６年 「いつか」に広域で備えよ」,http://www.iza.ne.jp/news/newsarticle/column/o...,... 【主張】 阪神淡路大震災から１７日で１６年がたつ。 ...,... 、大阪などは標識が目立たずほとんど知られていない。防災標識は規格統一すべきだ。 また...,iza：イザ！,NaN,Japan,NaN,Japanese,...,NaN,NaN,NaN,NaN,NaN,15-jan-11,11:57 em,NaN,NaN,NaN
2,15-Jan-2011 11:56PM,Weekend Football TV Times: Jan 15-17,http://www.straitstimes.com/BreakingNews/Sport...,Jan 15 Asian Cup: Group D - Iran v North Korea...,"Saudi Arabia v Japan (Live, mio TV Ch115 & HD ...",The Straits Times - Singapore,NaN,Singapore,NaN,English,...,NaN,NaN,NaN,NaN,NaN,15-jan-11,11:56 em,NaN,NaN,NaN
3,15-Jan-2011 11:52PM,Funde von Max von Oppenheim,http://www.express.de/express/regional/koeln/d...,... [ Schließen ] zurück 1 | 5 weiter Max von ...,. Seine letzte Nahostreise unternahm der Orien...,Express.de,NaN,Germany,Nordrhein-Westfalen regional,German,...,NaN,NaN,NaN,NaN,NaN,15-jan-11,11:52 em,NaN,NaN,NaN
4,15-Jan-2011 11:51PM,称阿拉克重水反应堆生产出氘化物 伊朗公布一项核进展,http://news.china.com.cn/rollnews/2011-01/16/c...,... 内容摘要: （记者 何光海 杜源江）伊朗代理外长兼原子能组织主席萨利希15日说，伊朗...,... 。他说：“阿拉克重水反应堆设施在氘化物方面已经实现自给自足。” 受伊朗政府邀请，阿尔...,中国网 - 新闻中心,NaN,Mainland China,Beijing,Chinese Simplified,...,NaN,NaN,NaN,NaN,NaN,15-jan-11,11:51 em,NaN,NaN,NaN


In [4]:
df.count()

Date                     853083
Headline                 853055
URL                      853061
Opening Text             810513
Hit Sentence             843419
Source                   853060
Influencer               312794
Country                  853060
Subregion                289776
Language                 853073
Reach                    853073
Desktop Reach            853073
Mobile Reach             853073
Twitter Social Echo      853073
Facebook Social Echo     853073
Reddit Social Echo       853073
National Viewership      853060
Engagement                   13
AVE                      853073
Sentiment                853060
Key Phrases              714101
Input Name               853073
Keywords                 853060
Twitter Authority             0
Tweet Id                      0
Twitter Id                    0
Twitter Client                0
Twitter Screen Name           0
User Profile Url              0
Twitter Bio                   0
Twitter Followers             0
Twitter 

In [5]:
# Only keeping English articles by making a filter that the language column must be English
filter = df['Language'].str.contains('English', na=False)
count=len(df[filter])
print(count)

288151


In [6]:
# Setting the dataframe to the created English filter
df=df[filter]

In [7]:
# Creating a filter to only use the articles the has Syria as a keyword
filter_syr=filter = df['Keywords'].str.contains('Syria', na=False)
count_s=len(df[filter_syr])
count_s

229654

In [8]:
# Making a dataframe with the Syria keyword filter 
df_filter = df[filter_syr]

In [9]:
df_filter.count()

Date                     229654
Headline                 229653
URL                      229654
Opening Text             200901
Hit Sentence             222552
Source                   229654
Influencer               130864
Country                  229654
Subregion                 63552
Language                 229654
Reach                    229654
Desktop Reach            229654
Mobile Reach             229654
Twitter Social Echo      229654
Facebook Social Echo     229654
Reddit Social Echo       229654
National Viewership      229654
Engagement                    0
AVE                      229654
Sentiment                229654
Key Phrases              227220
Input Name               229654
Keywords                 229654
Twitter Authority             0
Tweet Id                      0
Twitter Id                    0
Twitter Client                0
Twitter Screen Name           0
User Profile Url              0
Twitter Bio                   0
Twitter Followers             0
Twitter 

In [31]:
# Save the file
df_filter.to_csv('/Users/emiliatrulsson/Desktop/Syria Preprocess/syria_unhcr', index=False)

# Web Scraper Process for the UNHCR Links

Pre-requisite files:

- syria_unhcr, only English written articles with the keyword Syria from the original file provided by UNHCR
- ukraine_unhcr, the original file provided by UNHCR

For the web scraping process the same code is used for both groups, only  different files are used

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from datetime import timedelta

def get_html_content(url):
    try:
        response = requests.get(url, timeout=30)
        # check if the website allows web scraping
        robots_url = url.split('/')[0] + '//' + url.split('/')[2] + '/robots.txt'
        robots_response = requests.get(robots_url)
        if 'User-agent: *\nDisallow: /' in robots_response.text:
            return None
        else:
            return response.text
    except requests.exceptions.Timeout:
        print(f"Timeout occurred while fetching {url}, skipping...")
        return None
    except:
        return None

def save_html_to_file(html_content, file_name):
    with open(file_name, 'w') as html_file:
        html_file.write(html_content)

def delete_html_file(file_name):
    try:
        os.remove(file_name)
    except:
        pass

def get_headline_text_from_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    try:
        headline = soup.find('h1').text
    except AttributeError:
        headline = ''
    try:
        text_elements = soup.find_all('p')
        text = '\n'.join([element.text for element in text_elements])
    except AttributeError:
        text = ''
    return headline, text

def save_data_to_csv(data, file_name):
    df = pd.DataFrame(data, columns=['headline', 'text', 'URL'])
    df.to_csv(file_name, index=False)

def main():
    csv_file = '/Users/emiliatrulsson/Desktop/Preprocess/syria_unhcr.csv'
    df = pd.read_csv(csv_file, low_memory=False)
    urls = df['URL'].tolist()
    data = []
    start_time = time.time()
    for i, url in enumerate(urls):
        print(f"Processing URL {i+1}/{len(urls)}")
        html_content = get_html_content(url)
        if html_content is None:
            print(f"{url} does not allow web scraping")
            continue
        file_name = f"{url.split('/')[-1]}.html"
        if len(file_name) > 255:
            print(f"File name {file_name} is too long, skipping...")
            continue
        save_html_to_file(html_content, file_name)
        headline, text = get_headline_text_from_html(html_content)
        data.append({'headline': headline, 'text': text, 'URL': url})
        elapsed_time = time.time() - start_time
        avg_time_per_url = elapsed_time / (i+1)
        remaining_time = timedelta(seconds=(len(urls) - (i+1)) * avg_time_per_url)
        print(f"Estimated time remaining: {remaining_time}")
        delete_html_file(file_name)
    save_data_to_csv(data, '/Users/emiliatrulsson/Desktop/Preprocess/syria_scraped.csv ')


if __name__ == '__main__':
    main()